Using abcTau to fit ACFs for trials (Figure 2 from the paper). 

Three options to do that:
* use abcTau package for both ACF and fitting
* use ACF calculated before using acf function
* use ACF calculated before using iSTTC concat function

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 

import pickle
import re
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

from isttc.scripts.cfg_global import project_folder_path

In [2]:
dataset_folder = project_folder_path + 'synthetic_dataset\\'
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'

results_folder_abctau = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau_abctau\\'
datasave_path = results_folder_abctau + 'final_results\\'

### Load data (spike trains and calculated acf's)

In [ ]:
with open(dataset_folder + 'trials40_binned.pkl','rb') as f:
    data_binned = pickle.load(f)

trial_dict_binned = data_binned['trial_dict']
alphas_binned  = data_binned['alphas']
fr_values_binned  = data_binned['fr_values']
taus_ms_binned  = data_binned['tau_ms']
n_trials_binned  = data_binned['n_trials']
trial_lens_binned  = data_binned['trial_lens']

print(f'n spike trains {len(trial_dict_binned)}, trial_lens {trial_lens_binned[0]} ms')

In [ ]:
with open(results_folder + 'tau_pearsonr_trial_50ms_20lags_dict.pkl', "rb") as f:
    pearsonr_trial_avg_dict = pickle.load(f)

with open(results_folder + 'tau_isttc_trial_concat_50ms_20lags_dict.pkl', "rb") as f:
    sttc_trial_concat_dict = pickle.load(f)

print(f'len pearsonr_trial_avg_dict {len(pearsonr_trial_avg_dict)}')
print(f'len sttc_trial_concat_dict {len(sttc_trial_concat_dict)}')

In [3]:
lv_df = pd.read_pickle(results_folder + 'lv_df.pkl')
print(f'Lv loaded for n spike trains {len(lv_df)}')

lv_df

Lv loaded for n spike trains 100000


,unit_id,alpha,fr,tau_ms,lv
0,0,0.851099,7.741821,150.923515,1.307775
1,1,0.858098,4.394396,147.947233,1.345006
2,2,0.528805,8.587393,267.002290,1.068416
3,3,0.300692,6.976707,194.403797,1.055631
4,4,0.691458,0.950832,239.986489,1.357205
...,...,...,...,...,...
99995,99995,0.244627,2.672839,125.243251,1.178739
99996,99996,0.596987,5.754497,94.481295,1.353800
99997,99997,0.258915,5.604441,174.189672,1.064661
99998,99998,0.187620,1.219057,76.565797,1.306500


In [4]:
df_all = pd.read_pickle(results_folder + "summary_tau_all_long_df.pkl")

In [8]:
df_all.groupby(by='method').median()

,unit_id,tau,tau_lower,tau_upper,fit_r_squared,acf_decline,tau_ms,fr,alpha,tau_ms_true,tau_diff_abs,tau_diff_rel,ci_width,lv
method,,,,,,,,,,,,,,
acf_full,49972.0,3.311755,2.383958,4.005107,0.983834,1.0,165.587729,5.662966,0.509449,173.555183,18.803737,12.466582,0.974635,1.214061
isttc_full,49972.0,3.342355,2.410708,4.033805,0.984340,1.0,167.117761,5.662966,0.509449,173.555183,17.021454,11.321315,0.928842,1.214061
pearsonr_trial_avg,49972.0,2.812291,-1.107704,7.174618,0.548274,1.0,140.614571,5.662966,0.509449,173.555183,92.870516,58.097391,7.658720,1.214061
sttc_trial_concat,49972.0,2.694455,-0.825543,6.716205,0.562173,1.0,134.722759,5.662966,0.509449,173.555183,87.168539,55.061179,6.939604,1.214061


In [ ]:
df_all.keys()

### Summary df

In [17]:
directory = Path(datasave_path)
files = [file for file in directory.iterdir() if file.is_file()]
print(f'N files {len(files)}')

records = []
for file in files:
    # print(file.name)  
    # the last iteration is encoded in the name
    ind = file.name.find('steps') 
    final_step = int(file.name[ind+5] + file.name[ind+6])
    # het results 
    abc_results = np.load(file, allow_pickle=True)
    theta_accepted = abc_results[final_step-1]['theta accepted']
    tau = theta_accepted[0]

    match_unit_id = re.search(r"spike_train_(\d+)_", file.name)
    unit_id = int(match_unit_id.group(1))
    records.append({
        'unit_id': unit_id,
        'tau_mean': np.mean(tau),
        'tau_median': np.median(tau),
        'method': 'abctau_full',
    })

tau_df = pd.DataFrame(records)


N files 33


In [15]:
tau_df_merged = tau_df.merge(lv_df, how='left', on='unit_id')
tau_df_merged['tau_diff_abs'] = np.abs(tau_df_merged['tau_mean'] - tau_df_merged['tau_ms'])
tau_df_merged['tau_diff_rel'] = tau_df_merged['tau_diff_abs'] / tau_df_merged['tau_ms'] * 100
tau_df_merged.sort_values(by='tau_diff_rel')

,unit_id,tau_mean,tau_median,method,alpha,fr,tau_ms,lv,tau_diff_abs,tau_diff_rel
11,11,74.717406,75.122133,abctau_full,0.843413,9.268382,62.588591,1.488249,12.128815,19.378635
16,101,124.183587,121.830719,abctau_full,0.856319,7.000074,156.496852,1.361729,32.313266,20.647869
23,105,147.332822,142.900120,abctau_full,0.875132,7.171733,193.573821,1.370049,46.241000,23.888044
1,1,100.570520,99.500862,abctau_full,0.858098,4.394396,147.947233,1.345006,47.376713,32.022710
0,0,98.897887,96.508353,abctau_full,0.851099,7.741821,150.923515,1.307775,52.025629,34.471519
21,17,117.075113,114.187235,abctau_full,0.829656,0.647534,179.388791,1.376228,62.313679,34.736662
22,104,171.733041,169.080053,abctau_full,0.868956,7.789722,124.109195,1.384461,47.623846,38.372537
27,107,38.282272,37.677102,abctau_full,0.559739,2.729693,66.331006,1.489303,28.048733,42.286007
6,6,60.036541,57.989986,abctau_full,0.800260,7.613786,112.904124,1.317774,52.867583,46.825201
7,7,104.740170,106.605302,abctau_full,0.781671,7.862782,229.046160,1.246275,124.305990,54.271152
